In [1]:
import numpy as np
import ipyvolume as ipv
import ipywidgets as ipw
import scipy
from scipy.spatial.transform import Rotation as rot
from scipy.spatial.distance import cdist
import math
import matplotlib
import matplotlib.pyplot as plt
import random
from random import uniform

In [2]:
def cart2sph(v):
    x,y,z = v
    XsqPlusYsq = x**2 + y**2
    r = np.sqrt(XsqPlusYsq + z**2)                  # r
    elev = math.atan2(z,np.sqrt(XsqPlusYsq))        # theta
    az = math.atan2(y,x)                            # phi
    return np.array([r, elev, az])

def sph2cart(v):
    r, elev, az = v
    x = r * np.sin(elev) * np.cos(az)
    y = r * np.sin(elev) * np.sin(az)
    z = r * np.cos(elev)
    return np.array([x,y,z])

def angle(v1, v2):
    """ Returns the angle in degrees between vectors 'v1' and 'v2'"""
    v1_u = v1/np.linalg.norm(v1)
    v2_u = v2/np.linalg.norm(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))*180/np.pi

def random_point_on_circle(r):
    angle = np.random.uniform(0.,2.*np.pi)
    x = np.cos(angle)*r 
    y = np.sin(angle)*r
    return np.array([x,y])

def random_point_on_lattice(r):
    choices = np.array([[0,1],[0,-1],[1,0],[-1,0]])
    idx = np.random.choice(len(choices),1)
    return choices[idx]*r


def random_point_on_shpere(R):
    """ Returns points on a sphere with radius R, uniformly distributed"""
    radius = R*(np.random.uniform(0.,1.))**(1./3.)
    theta = np.arccos(np.random.uniform(-1.,1.))
    phi = np.random.uniform(0.,2.*np.pi)
    v = sph2cart([radius,theta,phi])
    return v

def regular_points_on_sphere(n,r):
    coords = []
    alpha = 4.0*np.pi*r*r/n
    d = np.sqrt(alpha)
    m_nu = int(np.round(np.pi/d))
    d_nu = np.pi/m_nu
    d_phi = alpha/d_nu
    count = 0
    for m in range (0,m_nu):
        nu = np.pi*(m+0.5)/m_nu
        m_phi = int(np.round(2*np.pi*np.sin(nu)/d_phi))
        for n in range (0,m_phi):
            phi = 2*np.pi*n/m_phi
            point = sph2cart([r,nu,phi])
            coords.append(point)
    return np.asarray(coords)

def random_point_on_cone(R,theta,prev):
    """ Returns random vector with length R and angle theta between previos one, uniformly distributed"""
    theta *=np.pi/180.
    v = prev/np.linalg.norm(prev)
    # find "mostly orthogonal" vector to prev
    a = np.zeros((3,))
    a[np.argmin(np.abs(prev))]=1
    # find orthonormal coordinate system {x_hat, y_hat, v}
    x_hat = np.cross(a,v)/np.linalg.norm(np.cross(a,v))
    y_hat = np.cross(v,x_hat)
    # draw random rotation 
    phi = np.random.uniform(0.,2.*np.pi) 
    # determine vector (random rotation + rotation with theta to guarantee the right angle between v,w)
    w = np.sin(theta)*np.cos(phi)*x_hat + np.sin(theta)*np.sin(phi)*y_hat + np.cos(theta)*v
    w *=R
    return w

def random_coords(N,model,theta):
    coords = np.zeros((int(N),3))
    coords[1]=[1,0,0]
    for i in range(2,int(N)):
        if model == 'freely_jointed':
            n = random_point_on_shpere(1.0)
            new = coords[i-1]+n
        if model == 'freely_rotating':
            prev = coords[i-2]-coords[i-1]
            n = random_point_on_cone(1.0,theta,prev)
            new = coords[i-1]+n
        coords[i]=new
    return coords


def random_coords_2D(N,type):
    coords = np.zeros((int(N),2))
    coords[1]=[1.,0.]
    for i in range(2,int(N)):
        if type=='continous':
            n = random_point_on_circle(1.0)
        elif type=='lattice':
            n = random_point_on_lattice(1.0)
        new = coords[i-1]+n
        coords[i]=new
    return coords

def random_walk(N):
    steps = np.random.randint(low = -1, high = 1, size = (N))*2 +1 
    random_trajectory = np.cumsum(steps)
    random_trajectory = np.concatenate([[0],random_trajectory])
    
    return random_trajectory

def calculate_overlaps(coords):
    d = cdist(coords,coords)
    i = np.arange(len(coords)).reshape(len(coords),1)
    u = cdist(i,i)
    q = np.where(np.logical_and(u > 1,d<1) , True, False)
    l = np.argwhere(q)
    l = np.unique(np.vstack((l[:,0],l[:,1])))
    return coords[l.astype(int)]
    

# Random walks
## 2D

In [3]:
n = ipw.widgets.IntText(
    value=10,
    description='N:',
    disabled=False,
continuous_update=False)

m = ipw.widgets.RadioButtons(
    options=['lattice', 'continous'],
    description='type:',
    disabled=False
)  

In [4]:
%matplotlib notebook
def random(n,type,draw_beads):
    a = random_coords_2D(n,type)
    plt.figure()
    plt.plot(a[:,0],a[:,1],color='black',linewidth=0.2)
    if draw_beads:
        plt.scatter(a[:,0],a[:,1],color='red')
    plt.show()
    return a

#2D
walk = ipw.interactive(random,{'manual': True},type=m,n=n,draw_beads=True);
display(walk)


interactive(children=(IntText(value=10, description='N:'), RadioButtons(description='type:', options=('lattice…

## 3D

In [5]:
def random(n,draw_beads):
    a = random_coords(n,'freely_jointed',0)
    ipv.figure(height=400,width=600)
    ipv.pylab.plot(a[:,0],a[:,1],a[:,2],color='black')
    if draw_beads:
        ipv.pylab.scatter(a[:,0],a[:,1],a[:,2],color='red')
    ipv.pylab.xyzlabel(' ',' ',' ')
    
    max = np.max(a)
    min = np.min(a)
    ipv.pylab.xlim(min, max)
    ipv.pylab.ylim(min, max)
    ipv.pylab.zlim(min, max)
    ipv.show()
    return a

#3D
walk = ipw.interactive(random,{'manual': True},n=n,draw_beads=True);
display(walk)

interactive(children=(IntText(value=10, description='N:'), Checkbox(value=True, description='draw_beads'), But…